In [2]:
# imports
import gzip
from matplotlib import pyplot
from collections import defaultdict
from datetime import datetime
# from google.colab import drive
import numpy as np
import matplotlib as plt
import random
import copy
import statistics
import urllib
import scipy.optimize
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import ast
from nltk.corpus import stopwords
from google.colab import drive
from sklearn.linear_model import LogisticRegression

In [3]:
drive.mount('/content/gdrive', force_remount=True)
# This is our working directory for assignment 2
!ls /content/gdrive/MyDrive/Assignment-2-CSE-158

Mounted at /content/gdrive
assignment2.pdf   Baseline.ipynb	      goodreads_reviews_spoiler.json.gz
BagOfWords.ipynb  exploratory_analysis.ipynb  tf-idf.ipynb


In [4]:
# parsing data from goodreads_reviews_spoiler.json.gz
def parseData(fname):
    for t in gzip.open(fname):
        t = t.decode("utf-8")
        t = t.replace('true', 'True')
        t = t.replace('false', 'False')
        yield eval(t)

dataset = list(parseData('/content/gdrive/MyDrive/Assignment-2-CSE-158/goodreads_reviews_spoiler.json.gz'))

In [5]:
nltk.download('stopwords')
allStopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# model: bag of words 
randomDataset = copy.deepcopy(dataset)
random.shuffle(randomDataset)

In [7]:
trainLength = 500000
testLength = 750000

In [8]:
dataTrain = randomDataset[:trainLength]
dataTest = randomDataset[trainLength:testLength]

In [9]:
wordCount = defaultdict(int)
totalWords = 0
punct = string.punctuation

In [10]:
for d in randomDataset[:trainLength]:
    for sentence in d['review_sentences']:
        t = sentence[1]
        t = t.lower() # lowercase string
        t = [c for c in t if not (c in punct)] # non-punct characters
        t = ''.join(t) # convert back to string
        words = t.strip().split() # tokenizes
        for w in words:
            # if w not in allStopwords:
#                     w = stemmer.stem(w)
              totalWords += 1
              wordCount[w] += 1

In [11]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [12]:
words = [w[1] for w in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [13]:
def feature(sentence):
    feat = [0]*len(wordSet)
    t = sentence
    t = t.lower() # lowercase string
    t = [c for c in t if not (c in punct)] # non-punct characters
    t = ''.join(t) # convert back to string
    words = t.strip().split() # tokenizes
    for w in words:
        if not (w in wordSet): continue
        feat[wordId[w]] += 1
    feat.append(1)
    return feat

In [ ]:
X = []
y = []

for d in randomDataset[:testLength]:
    for sentence in d['review_sentences']:
        X.append(feature(sentence[1]))
        y.append(sentence[0])

In [ ]:
Xtrain = X[:trainLength]
ytrain = y[:trainLength]
Xvalid = X[trainLength:testLength]
yvalid = y[trainLength:testLength]

In [ ]:
mod = linear_model.LogisticRegression(C=1, class_weight='balanced')
mod.fit(Xtrain, ytrain)

pred = mod.predict(Xvalid)

In [ ]:
acc = sum(pred == yvalid)/len(pred)
print(acc)

In [ ]:
# Balanced error rate 
TP_ = np.logical_and(pred, yvalid)
FP_ = np.logical_and(pred, np.logical_not(yvalid))
TN_ = np.logical_and(np.logical_not(pred), np.logical_not(yvalid))
FN_ = np.logical_and(np.logical_not(pred), yvalid)

In [ ]:
TP = sum(TP_)
FP = sum(FP_)
TN = sum(TN_)
FN = sum(FN_)

In [ ]:
1 - 0.5*(TP / (TP + FN) + TN / (TN + FP))